## Morning practical 2 day 2

Welcome to the second practical of today. Here, you will work on implementing regularised logistic regression, as well as implementing cross-validation on some data and making an ROC curve. First run the two cells below to set things up.



In [1]:
# run this cell to set things up
import ipywidgets as widgets, numpy as np, pandas as pd
from numpy.random import default_rng

%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import math
import seaborn as sns
from IPython.display import display, Markdown
from scipy.optimize import fmin_bfgs
from scipy.special import expit

In [2]:
# important functions
def mySigmoid(data):
    output = 1 / (1 + np.exp(-data))
    return output


# I have redefined the mySigmoid for numerical stability here.
# Why this? Well, with many parameters and large values, numerical precision for the power function becomes
# an issue. Read here: https://github.com/scipy/scipy/blob/91a279ecb05e7814e2787bfa618d46ad3e0af2be/scipy/special/_logit.h
# how scipy fixes that.
def mySigmoid(data):
    data = np.array(data)
    return expit(data)


def linAlgRegHypothesis(data, thetas):
    data = np.array(data)
    oneFeatToAdd = np.ones(len(data))
    newFeatArray = np.c_[oneFeatToAdd, data]
    # make sure thetas are always of the form np.array([[theta1], [theta2]]), i.e. column vector
    if thetas.ndim < 2:
        thetas = thetas[:, np.newaxis]
    predictions = newFeatArray @ thetas
    return predictions


def linAlgLogRegHypothesis(data, thetas):
    output = mySigmoid(linAlgRegHypothesis(data, thetas))
    return output


def costFuncLogReg(x, y, thetas):
    predictions = linAlgLogRegHypothesis(x, thetas)
    costsPerSample = -y * np.log(predictions) - (1 - y) * np.log(1 - predictions)
    totalCosts = np.nansum(1 / len(x) * costsPerSample)
    return totalCosts


def makeCrossValData(featureArray, y, k=10):
    """function to make splits into training and validation sets.
    Outputs two lists of length k, where each element is the indices of samples to train on for that fold,
    and the indices of samples to test on for that fold, respectively."""
    m = len(featureArray)
    # shuffle data
    shuffled_indices = np.random.permutation(m)
    shuffled_features = featureArray[shuffled_indices, :]  # assumes 2D array
    shuffled_labels = y[shuffled_indices]
    # see how many equal-sized sets you can make
    dataPerSplit = int(np.floor(m / k))
    dataPartitions = []
    counter = 0

    for i in range(0, k):
        # make a list of all the samples for each fold
        dataPartitions.append(list(range(counter, counter + dataPerSplit)))
        counter += dataPerSplit

    samplesEquallySplit = k * dataPerSplit
    if not samplesEquallySplit == m:
        # after making equal splits there will be samples left, i.e. you cannot always make k exactly evenly sized subsets.
        # randomly assign left over samples to folds after
        toDivide = m - samplesEquallySplit
        for extraSampleIndex in range(counter, counter + toDivide):
            # only assign to lists of samples that have the current minimum amount of samples
            currentSubsetSizes = np.array([len(subset) for subset in dataPartitions])
            assignTo = np.random.choice(
                np.where(currentSubsetSizes == np.min(currentSubsetSizes))[0]
            )
            dataPartitions[assignTo].append(extraSampleIndex)

    # Now make the final cross-validation set: make k sets, each set has (k-1)/k folds to train on, and 1 fold to test on.
    testSet = []
    trainSet = []
    for validationSetIndex in range(0, k):
        # put 1 fold in the test set
        testSet.append(dataPartitions[validationSetIndex])
        # put all other folds in the train set
        trainSet.append(dataPartitions.copy())
        trainSet[validationSetIndex].pop(validationSetIndex)
        # this line makes sure all training set indices are in one big list, rather than k-1 small lists.
        trainSet[validationSetIndex] = [
            item for sublist in trainSet[validationSetIndex] for item in sublist
        ]

    return shuffled_features, shuffled_labels, trainSet, testSet

## Regularisation

Regularisation is a method of automatically constraining how much your model can (over)fit on the training data. We add some factor (regularisation weight $\lambda$) times the sum of squares of the parameter (excluding the intercept ($\theta_0$) to the cost function. In this way, the model cannot pick extremely large values for the parameters, i.e. when you have 100 features, the model is forced to only have high $\theta$ parameters for those features that matter a lot for correct classification, while having extremely low or even 0 values for features that don't. Hence, regularisation also automatically selects features that are of importance to your problem: _feature selection_! (Strictly speaking, this holds only for when you penalise the absolute of the sum of the parameters, not when you penalise the square). Note that once you have trained the model and want to know the cost on the validation/test set, you should not use regularised cost: you care about your performance in the end (which you hope is better because you constrain the parameters during fitting).

* To get started, change your costFuncLogReg to have an extra argument `lambda_ = 0` ( _ because lambda is a keword for anonymous functions), that, if set to a value higher than 0, causes regularisation to be performed.
* Make sure to exclude the bias/intercept term ($\theta_0$) from this. By convention this is not regularised.
* While you are at it, also reorder the arguments to `thetas, x, y, lambda_=0` so it is easier to use another optimizer if we want to!

Hint:
* Remember that the regularised logistic regression cost function is:
![APicture](RegLogRegEq.PNG) <div>
You already had the first part implemented, you only need to add the second part!



In [3]:
# answer
def costFuncLogReg(thetas, x, y, lambda_=0):
    m = len(x)
    predictions = linAlgLogRegHypothesis(x, thetas)
    costsPerSample = -y * np.log(predictions) - (1 - y) * np.log(1 - predictions)
    # set the bias to 0 so you don't count it in regularisation.
    # It is convention not to regularise the bias/intercept
    thetas[0] = 0
    regCost = lambda_ / 2 * 1 / m * thetas.T @ thetas
    totalCosts = np.nansum(1 / m * costsPerSample) + regCost
    return totalCosts

## Changing gradient descent

The gradients should also change. Luckily, since all that's added is a plus term, the change is extremely minor:
![gradients_logreg](GradientsRegLogReg.PNG)

* Up to you to implement the changes in the `linAlgGradientDescent` function. Add another `lambda_ = 0` argument and change the gradients as needed. So `linAlgGradientDescent(x, y, thetas, alpha, lambda_ = 0, method = "linear")`.

In [4]:
# old function
def linAlgGradientDescent(x, y, thetas, alpha, method="linear"):
    possible_methods = ["linear", "logistic"]
    if method not in possible_methods:
        print(
            "Error! Wrong method given. Should be one of: "
            + str(possible_methods)
            + "\n Returning None!"
        )
        return
    m = len(x)
    ## all these shape/ndim calls are unnecessary if you input column vectors as you should.
    if thetas.ndim < 2:
        thetas = thetas[:, np.newaxis]
    if method == "linear":
        preds = linAlgRegHypothesis(x, thetas)
    else:
        preds = linAlgLogRegHypothesis(x, thetas)

    if preds.shape != (m, 1):
        preds = preds[:, np.newaxis]
    if y.shape != (m, 1):
        y = y[:, np.newaxis]
    errors = preds - y
    gradientSummation = errors.T @ np.c_[np.ones(len(errors)), x]
    finalGradientSteps = alpha / m * gradientSummation
    newThetas = thetas - finalGradientSteps.T
    return newThetas


# answer
# def linAlgGradientDescent(x, y, thetas, alpha, lambda_=0, method="linear"):
#     possible_methods = ["linear", "logistic"]
#     if method not in possible_methods:
#         print(
#             "Error! Wrong method given. Should be one of: "
#             + str(possible_methods)
#             + "\n Returning None!"
#         )
#         return
#     m = len(x)
#     ## all these shape/ndim calls are unnecessary if you input column vectors as you should.
#     if thetas.ndim < 2:
#         thetas = thetas[:, np.newaxis]
#     if method == "linear":
#         preds = linAlgRegHypothesis(x, thetas)
#     else:
#         preds = linAlgLogRegHypothesis(x, thetas)

#     if preds.shape != (m, 1):
#         preds = preds[:, np.newaxis]
#     if y.shape != (m, 1):
#         y = y[:, np.newaxis]
#     errors = preds - y
#     gradientSummation = errors.T @ np.c_[np.ones(len(errors)), x]
#     gradientSummation[1:] += lambda_ / m * thetas[1:]
#     finalGradientSteps = alpha / m * gradientSummation
#     newThetas = thetas - finalGradientSteps.T
#     return newThetas


def linAlgGradientDescent(x, y, thetas, alpha, lambda_=0, method="linear"):
    possible_methods = ["linear", "logistic"]
    if method not in possible_methods:
        print(
            "Error! Wrong method given. Should be one of: "
            + str(possible_methods)
            + "\n Returning None!"
        )
        return
    m = len(x)
    ## all these shape/ndim calls are unnecessary if you input column vectors as you should.
    if thetas.ndim < 2:
        thetas = thetas[:, np.newaxis]
    if method == "linear":
        preds = linAlgRegHypothesis(x, thetas)
    else:
        preds = linAlgLogRegHypothesis(x, thetas)

    if preds.shape != (m, 1):
        preds = preds[:, np.newaxis]
    if y.shape != (m, 1):
        y = y[:, np.newaxis]

    errors = preds - y  # shape (m,1)

    # build the augmented design matrix only for the gradient step
    # example shapes:
    #   errors.T: (1, m) like [[e1, e2, ..., em]]
    #   np.c_[np.ones(m), x]: (m, n+1) like [[1, x11, x12, ... x1n],
    #                                         [1, x21, x22, ... x2n],
    #                                         ...
    #                                         [1, xm1, xm2, ... xmn]]
    #   result: (1, n+1) like [[Σe_i, Σ(e_i*x_i1), Σ(e_i*x_i2), ... Σ(e_i*x_in)]]
    gradientSummation = errors.T @ np.c_[np.ones(len(errors)), x]

    # L2 regularization on non-bias terms (exclude the first column)
    # example shapes:
    #   gradientSummation[:, 1:]: (1, n)
    #   thetas[1:].T:             (1, n)
    gradientSummation[:, 1:] += lambda_ * thetas[1:].T

    finalGradientSteps = alpha / m * gradientSummation  # scale by alpha/m
    newThetas = thetas - finalGradientSteps.T  # (n+1,1) - (n+1,1)
    return newThetas


# Refactoring into two separate functions

Below, I have made one function called `computeGradients()` that computes and returns the gradients, and another function called `gradientDescentStep()` that takes a step using current thetas, those gradients, and an alpha value. In this way, we can use the first one if we want to use some other optimizer (which wants just the gradients), and the second one if we want to use gradient descent proper.

In [5]:
# refactor into separate functions
def computeGradients(thetas, x, y, lambda_=0, method="linear"):
    m = len(x)
    if thetas.ndim < 2:
        thetas = thetas[:, np.newaxis]
    if method == "linear":
        preds = linAlgRegHypothesis(x, thetas)
    else:
        preds = linAlgLogRegHypothesis(x, thetas)

    if preds.shape != (m, 1):
        preds = preds[:, np.newaxis]
    if y.ndim < 2:
        y = y[:, np.newaxis]
    errors = preds - y
    gradientSummation = errors.T @ np.c_[np.ones(len(errors)), x]
    unregularisedGradients = 1 / m * gradientSummation
    regularisedGradients = np.ravel(unregularisedGradients)
    regularisedGradients[1:] = (
        regularisedGradients[1:] + lambda_ / m * np.ravel(thetas)[1:]
    )
    # print("final regularised gradients:")
    # print(regularisedGradients)
    return regularisedGradients


def gradientDescentStep(thetas, gradients, alpha):
    if thetas.ndim < 2:
        thetas = thetas[:, np.newaxis]
    if gradients.ndim < 2:
        gradients = gradients[:, np.newaxis]
    newThetas = thetas - alpha * gradients
    return newThetas

## Loading in some data 

Let's look at the Pima Indians dataset, which contains information on multiple clinical variables and whether or not patients have diabetes. The below code loads in the data. I am using pandas since it has a nice .describe() method for DataFrame that shows you information about the data. Up to you to investigate this data somewhat:

* Are there any NaNs in the data?
* Are there other values that seem circumspect? Name 2 examples. How many of these circumspect values are there in these features?
* How many cases and controls are there? Is this a balanced dataset?

Hint(s):
* Use the `.describe` method of the dataframe to help you answer these questions.
* You can slice a dataframe using `df.loc[df["colName"] < 12, :]`, which corresponds to getting you only rows for which the values in column _colName_ are less than 12.

In [6]:
diabetesData = pd.read_csv("PimaIndiansDiabetes.csv")

# answer
display(diabetesData.describe())
# there are no NaNs
print("nr of NaNs:")
np.sum(np.isnan(diabetesData))
# there are indeed circumspect values. A SkinThickness of 0 or glucose level of 0 or
# even a BMI of 0 is probably not correct.
# we'll need to do something about this.
print("No. cases BMI not measured:")
print(len(diabetesData.loc[diabetesData["BMI"] <= 0, :]))
print("No. cases SkinThickness not measured:")
print(len(diabetesData.loc[diabetesData["SkinThickness"] <= 0, :]))
print("No. cases Insulin not measured:")
print(len(diabetesData.loc[diabetesData["Insulin"] <= 0, :]))
# etc.

# cases and controls
print("Nr. of cases: ")
print(len(diabetesData.loc[diabetesData["Outcome"] == 1, :]))
print("Nr. of controls: ")
print(len(diabetesData.loc[diabetesData["Outcome"] == 0, :]))
print("The dataset is unbalanced")


FileNotFoundError: [Errno 2] No such file or directory: 'PimaIndiansDiabetes.csv'

## Cleaning up the dataset

# Note: this was removed from the practical. You can skip this.

The dirty secret of ML is that you spend most of your time cleaning data. So you'll have to spend some time on that here. Do the following:

* Replace the 0 values with `np.nan` (**Note**: be aware that you shouldn't do this for all columns. Think about it.)
* Use [sklearn.impute.KNNImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html) to impute values that are missing for those columns where you inserted NaNs. Those who have followed the BiBC Essentials Course might remember K-Nearest Neighbour clustering. This function determines the (by default) 5 most similar samples (based on data that is _not_ missing) and sets the bmi/glucose level, etc. to the mean of their values. Euclidean distance is used. We will discuss K-Nearest Neighbour clustering in two days. For now, you can just use it. To do so, use `a = KNNImputer(missing_values = np.nan)` followed by `imputedData = a.fit_transform(nonImputedData)`.
* Note that this turns the DataFrame into a numpy array: this is not a problem but it's good to know.
* Mean-normalise (i.e. subtract the mean and divide by the standard deviation) the features using the function provided below. This should be done on all the data except the labels.
* Put the class into a `np.array` (a column vector) called `diabetesClassLabels`

In [ ]:
from sklearn.impute import KNNImputer


def createNormalisedFeatures(featureArray, mode="range", printit=False):
    if printit:
        print(featureArray)
    featureMeans = np.mean(featureArray, axis=0, keepdims=True)
    if printit:
        print(featureMeans)
    if printit:
        print(featureArray - featureMeans)
    if mode == "range":
        featureRanges = np.max(featureArray, axis=0, keepdims=True) - np.min(
            featureArray, axis=0, keepdims=True
        )
        # broadcasting in action:
        normalisedFeatures = (featureArray - featureMeans) / featureRanges
        return [normalisedFeatures, featureMeans, featureRanges]
    elif mode == "SD":
        featureSDs = np.std(featureArray, axis=0, keepdims=True)
        # broadcasting in action:
        normalisedFeatures = (featureArray - featureMeans) / featureSDs
        return [normalisedFeatures, featureMeans, featureSDs]


# answer
# replace 0 with Nan. Don't do this for pregnancies: for all you know it could be a real 0 there.
# also not for having diabetes or not, of course!
dfSubset = diabetesData.iloc[:, 1:-1]
display(dfSubset.head())
dfSubsetNan = dfSubset.replace(0, value=np.nan)
display(dfSubsetNan.head())
# impute values
imputer = KNNImputer(missing_values=np.nan)
dfSubsetNan = imputer.fit_transform(dfSubsetNan)

# add back the pregnancies column
featuresDiabetesNotNorm = np.append(
    dfSubsetNan, np.array(diabetesData["Pregnancies"])[:, np.newaxis], axis=1
)

# normalise

normDiabFeats, meansDiabFeats, standardDevsDiabFeats = createNormalisedFeatures(
    featuresDiabetesNotNorm, "SD"
)
for index, row in enumerate(normDiabFeats[0:10]):
    print(f"Data row {index} : {row}")
for colname, mean in zip(diabetesData.columns, np.mean(normDiabFeats, axis=0)):
    print(f"Mean of {colname}: {np.round(mean)}")
diabetesClassLabels = np.array(diabetesData["Outcome"])[:, np.newaxis]

# Crappier data

Hey, you know how you just cleaned some data? Good on you! However, for demonstration purposes to show regularisation in action, it's actually good to have some data that you might overfit on. To do that, I've asked your good friend ChatGPT-4O, and it modified the dataset to include:
* Correlated (Redundant) Features: Duplicates of existing features with slight noise.
* Irrelevant Noisy Features: Five random noise columns.
* Polynomial and Interaction Features: Nonlinear relationships (e.g., Glucose × Insulin).
* Weakly Predictive Features: Slightly correlated to the target variable with added noise.
* Scaling Inconsistencies: A feature (BMI) scaled by 1000.

We'll work with that, for demonstration purposes. 

In [ ]:
pima_messy = pd.read_csv("PimaIndiansDiabetes_MessedUpNoise.csv")

# Ensure the dataset has an 'Outcome' column
# Generate 50 'Weak_Signal' columns efficiently
if "Outcome" in pima_messy.columns:
    weak_signals = pima_messy["Outcome"].values[
        :, np.newaxis
    ] * 0.05 + np.random.uniform(-1, 1, (pima_messy.shape[0], 50))
    weak_signal_columns = [f"Weak_Signal_{i}" for i in range(1, 51)]

    # Generate 200 random noise columns efficiently
    random_noise = np.random.uniform(-1, 1, (pima_messy.shape[0], 200))
    noise_columns = [f"Random_Noise_{i}" for i in range(1, 201)]

    # Convert to DataFrames and concatenate efficiently
    pima_messy = pd.concat(
        [
            pima_messy,
            pd.DataFrame(weak_signals, columns=weak_signal_columns),
            pd.DataFrame(random_noise, columns=noise_columns),
        ],
        axis=1,
    )


# Define the normalization function
def createNormalisedFeatures(featureArray, mode="range", printit=False):
    featureMeans = np.mean(featureArray, axis=0, keepdims=True)
    if mode == "range":
        featureRanges = np.max(featureArray, axis=0, keepdims=True) - np.min(
            featureArray, axis=0, keepdims=True
        )
        normalisedFeatures = (featureArray - featureMeans) / featureRanges
        return [normalisedFeatures, featureMeans, featureRanges]
    elif mode == "SD":
        featureSDs = np.std(featureArray, axis=0, keepdims=True)
        normalisedFeatures = (featureArray - featureMeans) / featureSDs
        return [normalisedFeatures, featureMeans, featureSDs]


# Extract the 'Pregnancies' and 'Outcome' columns separately
pregnancy_col = pima_messy["Pregnancies"].values[:, np.newaxis]
outcome_col = pima_messy["Outcome"].values[:, np.newaxis]
# print('Columns in data:\n')
# print('\n'.join(pima_messy.columns),'\n')

# Replace 0 with NaN for imputation (excluding 'Pregnancies' and 'Outcome')
dfSubsetNan_messy = pima_messy.drop(columns=["Pregnancies", "Outcome"]).replace(
    0, np.nan
)

# Impute missing values using KNN imputer
imputer = KNNImputer(missing_values=np.nan)
dfSubsetNan_messy = imputer.fit_transform(dfSubsetNan_messy)

# Add back the 'Pregnancies' column
featuresMessyNotNorm = np.append(dfSubsetNan_messy, pregnancy_col, axis=1)

# Normalize using standard deviation method
normMessyFeats, meansMessyFeats, standardDevsMessyFeats = createNormalisedFeatures(
    featuresMessyNotNorm, "SD"
)

# Store class labels
messyClassLabels = outcome_col

print("final data first 5 rows:")
display(normMessyFeats[:5, :10])
display(messyClassLabels[:5])
n_parameters_messy = normMessyFeats.shape[1]
print(f"n_parameters_messy: {n_parameters_messy}")

## Testing your new functions' mettle I

Okay, now we can train regularised logistic regression on this data. Let's **use lambda values of 0, 0.5, 1, 5, 10, 100, 1000 and 10000**. We'll downsample the data so we have equal amounts of the positive and negative class, and train the classifier on 80% of the training data while testing on 20% held-out data (normally we'd use cross-validation but let's not put that extra level of complication in here as well). 

The visualisation of a decision boundary/what has been learned is somewhat complex: we can't just draw some boundary in 2D as our data isn't 2D but 8D.
We'll reduce the dimensionality to two dimensions using PCA, and then show in those two dimensions which points are positive or negative for diabetes, and what the classifier predicts everywhere in that plane. This is done for you. We'll talk about dimensionality reduction on the last day of this week. For now, know that, by its nature, dimensionality reduction will lose some of the true differences in your data, so visualisation of the decision boundary in this 2D space is bound to be an approximation, and cannot capture completely what your classifier is doing (as it's separating things in 8 dimensions rather than 2)! 



Your job:

* Downsample the normalised messy Diabetes Data (`normMessyFeats`): remove random rows of the controls so you have equal # of non-diabetes and diabetes cases. You could use `np.random.choice(a=rowIndicesOfoRowsThatDon'tHaveDiabetes, size = howManySamplesNeedToBeRemoved, replace = False)`, where you then remove (`np.delete()` can be useful) those rows from the feature and class label array. You'll probably also need `np.ravel(messyClassLabels)` and `np.where()`. <br> Save the new data as `equalClassSizeDiabetesData` and `equalClassSizeClassLabels` for the labels.
Hints: 
* `np.where` returns a tuple, of which you need the first element.
* Note that you can always insert a new cell above or below the current one for testing or debugging using `escape + a` or `escape + b`.

In [ ]:
# make sure everyone gets the same split
np.random.seed(42)


# downsample the data


# answer

# downsampling
whereClassLabelsAreNegative = np.where(np.ravel(messyClassLabels == 0))[0]
nrPositiveLabels = len(messyClassLabels) - len(whereClassLabelsAreNegative)
nrOfRowsToRemove = len(whereClassLabelsAreNegative) - nrPositiveLabels
rowIndicesToRemove = np.random.choice(
    a=whereClassLabelsAreNegative, size=nrOfRowsToRemove, replace=False
)

equalClassSizeDiabetesData = np.delete(
    np.array(normMessyFeats), rowIndicesToRemove, axis=0
)
equalClassSizeClassLabels = np.delete(messyClassLabels, rowIndicesToRemove, axis=0)

# check that it worked
print(len(np.where(equalClassSizeClassLabels == 0)[0]))
print(len(np.where(equalClassSizeClassLabels == 1)[0]))


## Testing your new functions' mettle II

* Make a list of the lambda values to train on (`lambdaValues`; **use lambda values of 0, 0.5, 1, 5, 10, 100, 1000 and 10000**), an empty list to store the test cost in (`testCostList`), and a list for the final thetas after gradient descent (`finalThetaList`).
* Randomly sample 80% of the data you produced above for training, and save the rest for testing. **_Code for this is given below!_**.
* Now make a `for`-loop that loops over the different lambdaValues.
* In that loop, make another loop that performs 300 gradient descent steps with an alpha of 0.2 on `trainDataDiabetes`.
* After that's done, calculate the cost on `testDataDiabetes` **without regularisation (lambda of 0)**. Remember: you don't use the regularisation parameter in the final predictions, because you use it _during training_ to prevent overfitting, and then want to know how well you really do on the test data. 
* Append the result to the `testCostList`.
* Finally, look at the DataFrame containing the theta parameters found for the different values of lambdas, and the cost calculated on the test set (code to make it is given below). What do you see? 

Hints:
* There are many steps here. If you get stuck on one, ask a question or look at the answers to see how to do that step.

In [ ]:
np.random.seed(900)
startThetas = np.array([0] * (n_parameters_messy + 1))[:, np.newaxis]
nSteps = 500
alpha = 0.2

# making lists


#     code for dividing into 80% and 20%

nrSamplesToTake = int(np.ceil(0.8 * np.sum(equalClassSizeClassLabels == 0)))
negativeSampleIdxTrain = np.random.choice(
    np.arange(0, np.sum(equalClassSizeClassLabels == 0)),
    size=nrSamplesToTake,
    replace=False,
)
positiveSampleIdxTrain = np.random.choice(
    np.arange(0, np.sum(equalClassSizeClassLabels == 1)),
    size=nrSamplesToTake,
    replace=False,
)
positiveSamplesTrain, positiveClassLabelsTrain = (
    equalClassSizeDiabetesData[np.ravel(equalClassSizeClassLabels) == 1, :][
        positiveSampleIdxTrain, :
    ],
    equalClassSizeClassLabels[np.ravel(equalClassSizeClassLabels) == 1, :][
        positiveSampleIdxTrain, :
    ],
)
negativeSamplesTrain, negativeClassLabelsTrain = (
    equalClassSizeDiabetesData[np.ravel(equalClassSizeClassLabels) == 0, :][
        negativeSampleIdxTrain, :
    ],
    equalClassSizeClassLabels[np.ravel(equalClassSizeClassLabels) == 0, :][
        negativeSampleIdxTrain, :
    ],
)
trainDataDiabetes = np.vstack([positiveSamplesTrain, negativeSamplesTrain])
trainClassLabelsDiabetes = np.vstack(
    [positiveClassLabelsTrain, negativeClassLabelsTrain]
)


negativeSampleIdxTest = np.array(
    [
        i
        for i in np.arange(0, np.sum(equalClassSizeClassLabels == 0))
        if i not in negativeSampleIdxTrain
    ]
)
positiveSampleIdxTest = np.array(
    [
        i
        for i in np.arange(0, np.sum(equalClassSizeClassLabels == 1))
        if i not in positiveSampleIdxTrain
    ]
)
positiveSamplesTest, positiveClassLabelsTest = (
    equalClassSizeDiabetesData[np.ravel(equalClassSizeClassLabels) == 1, :][
        positiveSampleIdxTest, :
    ],
    equalClassSizeClassLabels[np.ravel(equalClassSizeClassLabels) == 1, :][
        positiveSampleIdxTest, :
    ],
)
negativeSamplesTest, negativeClassLabelsTest = (
    equalClassSizeDiabetesData[np.ravel(equalClassSizeClassLabels) == 0, :][
        negativeSampleIdxTest, :
    ],
    equalClassSizeClassLabels[np.ravel(equalClassSizeClassLabels) == 0, :][
        negativeSampleIdxTest, :
    ],
)
testDataDiabetes = np.vstack([positiveSamplesTest, negativeSamplesTest])
testClassLabelsDiabetes = np.vstack([positiveClassLabelsTest, negativeClassLabelsTest])


# your looping code, performing gradient descent for each lambda and
# calculating the cost on the test set after it's done, should go here:


#     code to make a final DataFrame to show what happens:
#     Uncomment all this code at once by selecting it and pressing Ctrl + /

# finalThetas = [np.ravel(elem) for elem in finalThetas]
# dataFrame = pd.DataFrame(np.c_[np.vstack(finalThetas), np.array(testCostList)])
# columnNames = ["theta_" + str(elem) for elem in list(range(len(startThetas)))]
# columnNames.append("testSetCost")
# dataFrame.columns = columnNames
# dataFrame.set_index(lambdaValues, inplace = True)
# display(dataFrame)

# answer

# making lists
lambdaValues = np.array([0.0, 0.5, 1, 5, 10, 100, 1000, 10000])
testCostList = []
finalThetaList = []
finalThetaListBFGS = []
testCostListBFGS = []

# also keep thetas for each run along descent, this is extra in the answer. You only need to keep the final theta
thetasAlongDescent = []
thetasAlongDescentBFGS = []
for lambda_ in lambdaValues:
    print("lambda_value: " + str(lambda_))
    thetas = startThetas
    # print(thetas)
    thetasAlongDescent.append([thetas])
    for step in range(0, nSteps):
        gradients = computeGradients(
            thetas, trainDataDiabetes, trainClassLabelsDiabetes, lambda_=lambda_
        )
        if step % 100 == 0:
            print(gradients[:5])
        thetas = gradientDescentStep(thetas, gradients, alpha)
        thetasAlongDescent[-1].append(thetas)
    # once done with gradient descent, save resulting cost and thetas.
    finalThetaList.append(thetas)
    testCostList.append(
        costFuncLogReg(thetas, testDataDiabetes, testClassLabelsDiabetes, lambda_=0)
    )

    # Also calculate BFGS thetas for reference
    thetasBFGS = fmin_bfgs(
        costFuncLogReg,
        np.ravel(startThetas),
        computeGradients,
        (trainDataDiabetes, trainClassLabelsDiabetes, lambda_),
        retall=True,
    )
    thetasAlongDescentBFGS.append(thetasBFGS[1])
    finalThetaListBFGS.append(thetasBFGS[0])
    testCostListBFGS.append(
        costFuncLogReg(
            thetasBFGS[0], testDataDiabetes, testClassLabelsDiabetes, lambda_=0
        )
    )

# showing the results for gradient descent
finalThetas = [np.ravel(elem[-1]) for elem in thetasAlongDescent]
finalThetas = [np.ravel(elem) for elem in finalThetaList]
dataFrame = pd.DataFrame(np.c_[np.vstack(finalThetas), np.vstack(testCostList)])
columnNames = ["theta_" + str(elem) for elem in list(range(len(startThetas)))]
columnNames.append("testSetCost")
dataFrame.columns = columnNames
dataFrame.set_index(lambdaValues, inplace=True)
display(dataFrame)

# showing the result for min_bfgs
finalThetas = [np.ravel(elem) for elem in finalThetaListBFGS]
dataFrame = pd.DataFrame(np.c_[np.vstack(finalThetas), np.vstack(testCostListBFGS)])
columnNames = ["theta_" + str(elem) for elem in list(range(len(startThetas)))]
columnNames.append("testSetCost")
dataFrame.columns = columnNames
dataFrame.set_index(lambdaValues, inplace=True)
display(dataFrame)


## Regularised logistic regression results

If all goes well, you will see that the cost on the test set is lowest when using $\lambda > 0$. Isn't that grand!? Not too strange when we all but forced overfitting by adding lots of uninformative features.  Spectacularly unconvincing example notwithstanding: in general, for unregularised classification, you run the risk of tuning the parameters _too specifically_ to the values in the training set, which increases the cost on the unseen test set. Regularisation guards against this by penalising too large parameters.

## Classifier performance

We've talked in the lectures about the performance of a classification algorithm. We want to know the true positive rate and false positive rates for a given threshold, but also the classifier's performance over a range of thresholds. It is not too difficult to make a ROC curve yourself. Let's do that now for the best classifier (with the lowest mean cost on the test set).

Up to you to:
* Make a range of 200 thresholds (from 1 to 0) for saying something is the positive set (use `np.linspace` for this).
* Make two empty lists: `truePositiveRates` and `trueNegativeRates`.
* Make predictions on the `testDataDiabetes` using the best set of learned thetas (which you can manually select or get from `finalThetas` using the index of the minimum element in `testSetCostList`).
* Make a for loop over the different thresholds you defined. Within that loop:
    * Turn the predictions into class labels using `np.where` and the current threshold value.
    * Calculate the true positive rate (sensitivity/recall) and append it to the list.
    * Calculate the true negative rate and append it to the `trueNegativeRates` list.
* Finally make a plot of the sensitivity (true positive rate) on the y-axis and 1-specificity (1-TNR) on the x-axis. (use `fig, ax = plt.subplots()` and `ax.plot()`). Don't forget to set the axis labels and a title!

See the relevant excerpt from the slide below, and look [here](https://glassboxmedicine.com/2019/02/23/measuring-performance-auc-auroc/) for more explanation if you want it! <br> ![SensitivityAndSpecificity](SensitivityAndSpecificity.PNG)

In [ ]:
# answer
thresholdRange = np.linspace(1, 0, 200)
truePositiveRate = []
trueNegativeRate = []
bestThetas = finalThetaList[np.nanargmin(testCostList)]
predictionsBest = linAlgLogRegHypothesis(testDataDiabetes, bestThetas)

for threshold in thresholdRange:
    classLabelsThisThreshold = np.ravel(np.where(predictionsBest <= threshold, 0, 1))
    truePositive = np.sum(
        classLabelsThisThreshold[np.ravel(testClassLabelsDiabetes == 1)] == 1
    )
    falsePositive = np.sum(
        classLabelsThisThreshold[np.ravel(testClassLabelsDiabetes == 0)] == 1
    )
    trueNegative = np.sum(
        classLabelsThisThreshold[np.ravel(testClassLabelsDiabetes == 0)] == 0
    )
    falseNegative = np.sum(
        classLabelsThisThreshold[np.ravel(testClassLabelsDiabetes == 1)] == 0
    )
    TPR = truePositive / (truePositive + falseNegative)
    TNR = trueNegative / (trueNegative + falsePositive)
    truePositiveRate.append(TPR)
    trueNegativeRate.append(TNR)

oneMinusSpecificity = [1 - elem for elem in trueNegativeRate]
fig, ax = plt.subplots(figsize=(10, 10))
ax.plot(oneMinusSpecificity, truePositiveRate)
ax.set_title("ROC curve for best thetas")
ax.set_xlabel("1-Specificity")
ax.set_ylabel("Sensitivity")
# ax.margins(x=0, y=0)

## What I'd like you to remember here:
* What regularisation is, and how it works (penalising large weights for parameters, thereby forcing the algorithm to focus on those that really give it a lot of _bang for its buck_ and decreasing overfitting)
* How to implement regularisation, and how the parameter $\lambda$ affects it
* How to do some basic cleaning on a dataset, and what _the idea_ of imputation is (specifically of a KNNImputer)
* How to make a ROC plot, and what exactly is depicted on it, as well as why we might want to compare something like ROC AUC between classifiers, rather than accuracy. Note also that ROC AUC also compares wholly unrealistic thresholds (we would never use a threshold where we just say that everyone is positive or negative, say), so more clever comparison metrics exist. Note also that the ROC AUC does not, in and of itself, say much about out-of-domain generalisation, which might be most important in the real world!

## Final words

Congratulations. You've implemented regularised logistic regression on a real dataset (that you cleaned up yourself) and made your own ROC curve. We'll now move on to multiclass logistic regression and then to neural networks!

## Survey
"I want a Survey, hey! Giving feedback for the very first time. I want a su-u-u-u-rvey, got some feedback, on my mi-i-i-n-d". Thanks Weird Al, [very cool](https://www.youtube.com/watch?v=notKtAgfwDA). Here you go: [clickety-click](https://docs.google.com/forms/d/e/1FAIpQLSfaeqtRTz5KMqcmxQuOI5GYWHMejjh5_yuiCNSnNblpdKb0hQ/viewform?usp=sf_link).